# Basis Swap Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-ir-swap-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

In [1]:
import refinitiv.dataplatform as rdp

%run ".\..\..\credentials.ipynb"

session = rdp.PlatformSession(
    APP_KEY,
    rdp.GrantPassword(
        RDP_LOGIN,
        RDP_PASSWORD
    )
)

session.open()
endpoint = rdp.Endpoint(session, "https://api.edp.thomsonreuters.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Valuing a Cross Currency Basis Swap

This sample value an existing basis swap defined leg by leg for a complete control of its term and conditions. The 'fields' requested are:
- the market value in cash (CleanMarketValueInDealCcy, MarketValueInDealCcy, AccruedAmountInDealCcy)
- their equivalent in in percent (AccruedPercent,CleanPricePercent,DirtyPricePercent,CleanPricePercent)
- the PV01 (PV01Bp,PV01AmountInDealCcy)
- the names of the zero-coupon curves used (DiscountCurveName,ForwardCurveName)
- the error information (ErrorCode,ErrorMessage)


In [4]:
request_body = {
    
    "fields" : ["InstrumentTag","MarketValueInDealCcy","DirtyPricePercent","FixedRatePercent","SpreadBp","PV01Bp",\
                "PV01AmountInDealCcy","DiscountCurveName","ForwardCurveName","ErrorCode","ErrorMessage"],
    
    "universe" : [
        {
            "instrumentType":"Swap",
            "instrumentDefinition": {
                "instrumentTag":"CCSwap-LIBOR6M-EURIBOR6M-5Y",
                "startDate":"2020-01-28",
                "tenor":"5Y",
                "legs":[
                {
                    "direction":"Paid",
                    "interestType":"Float",
                    "interestPaymentFrequency":"SemiAnnual",
                    "interestCalculationMethod":"Dcb_Actual_360",
                    "notionalCcy":"USD",
                    "indexName":"LIBOR",
                    "indexTenor":"6M",
                    "SpreadBp":0.0
                },
                {
                    "direction":"Received",
                    "interestType":"Float",
                    "interestPaymentFrequency":"SemiAnnual",
                    "interestCalculationMethod":"Dcb_Actual_360",
                    "notionalCcy":"EUR",
                    "indexName":"EURIBOR",
                    "indexTenor":"6M",
                    "SpreadBp":-10.0
                }]
            }
        }],
    
    "pricingParameters": {
        "valuationDate": "2020-10-20T00:00:00Z",
    },
        
    "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentTag'}, {'type': 'Float', 'name': 'MarketValueInDealCcy'}, {'type': 'Float', 'name': 'DirtyPricePercent'}, {'type': 'Float', 'name': 'FixedRatePercent'}, {'type': 'Float', 'name': 'SpreadBp'}, {'type': 'Float', 'name': 'PV01Bp'}, {'type': 'Float', 'name': 'PV01AmountInDealCcy'}, {'type': 'String', 'name': 'DiscountCurveName'}, {'type': 'String', 'name': 'ForwardCurveName'}, {'type': 'String', 'name': 'ErrorCode'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [['CCSwap-LIBOR6M-EURIBOR6M-5Y', 1001147.8538908, 100.11478538908, None, 0.0, 0.273044542085845, 27.3044542085845, 'USD 6M disc. USD 6M curve', 'USD 6M disc. USD 6M curve', '', ''], ['CCSwap-LIBOR6M-EURIBOR6M-5Y', 1073694.54732269, 100.095248664311, None, -10.0, 0.283475560402366, 30.4076534665513, 'USD 3M disc. EUR 3M curve', 'EUR 6M disc. EUR 6M curve', '', '']]}


In [3]:
import pandas as pd

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,MarketValueInDealCcy,DirtyPricePercent,FixedRatePercent,SpreadBp,PV01Bp,PV01AmountInDealCcy,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,BasisSwap-EURIBOR-3M6M-5Y,1.001148e+06,100.114785,None,0.0,0.273045,27.304454,USD 6M disc. USD 6M curve,USD 6M disc. USD 6M curve,,
1,BasisSwap-EURIBOR-3M6M-5Y,1.073695e+06,100.095249,None,-10.0,0.283476,30.407653,USD 3M disc. EUR 3M curve,EUR 6M disc. EUR 6M curve,,
